In [2]:
# Подгрузим необходимые для работы библиотеки
import glob
import numpy as np 
import pandas as pd 
import scipy
import copy
from random import *
from scipy import signal, fftpack
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from mpl_toolkits.mplot3d import Axes3D
from random import random
from numpy import array
from numpy import cumsum
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
import seaborn as sns
%matplotlib inline
with pd.option_context("display.max_rows", 10, "display.max_columns", 20):
    print(pd.get_option("display.max_rows")) 
    print(pd.get_option("display.max_columns"))
FN = 1
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import GRU, LSTM
from keras.layers import Bidirectional, Dense
from keras.models import Sequential

ModuleNotFoundError: No module named 'pandas'

In [ ]:
cd ..

# Class that save data

In [3]:
class SOP:
    control_results = None
    smoothed_control_results = None
    hord_shift = 145
    pc_shift = 50 
    pc_schemes = [3,4,5,6,9,10]
    hord_schemes = [1,2,7,8,11,12,13,14,15,16]
    transverse_defects_columns = [7,8,15,16]
    longitudinal_defects_set = None # Список тактов отслеживающих продольные дефекты
    transverse_defects_set = None # Список тактов отслеживающих поперечные дефекты
    
    def __init__(self, filepath="SOP1.txt"):
        self.filepath = filepath
        
    def read_control_results(self):
        # В рамках текущего модуля пока избавимся от первого столбца с номерами измерений, пока в них нет необходимости
        splited_info = ""
        with open(self.filepath, mode="r", encoding="cp1251") as f:
            splited_info = f.readline()
        self.control_results=pd.read_csv(self.filepath, encoding="cp1251", delimiter = " ", header = None, skiprows = 1)
        self.control_results = self.control_results.drop(columns=[0])
        print(self.control_results.shape[0])
        splited_info = splited_info.split(' ')
        self.date = splitedУВО_info[1]
        self.time = splited_info[2]
        self.temperature = splited_info[3]
        
    def fix_shift_issue(self):
        self.control_results = self.control_results.iloc[:1024]
        self.smoothed_control_results = self.smoothed_control_results.iloc[:1024]
        for scheme in self.pc_schemes:
            self.control_results[scheme] = np.roll(self.control_results[scheme], -self.pc_shift)
            self.smoothed_control_results[scheme] = np.roll(self.smoothed_control_results[scheme], -self.pc_shift)
        
        for scheme in self.hord_schemes:
            self.control_results[scheme] = np.roll(self.control_results[scheme], -self.hord_shift)
            self.smoothed_control_results[scheme] = np.roll(self.smoothed_control_results[scheme], -self.hord_shift)
    
    def smooth_control_results(self):
        self.smoothed_control_results = copy.deepcopy(self.control_results)
        for cont_res in self.control_results:
            self.smoothed_control_results[cont_res] = savgol_filter(self.control_results[cont_res], 15, 3)
    
    def split_by_defects(self):
        self.longitudinal_defects_set = self.smoothed_control_results.drop(self.transverse_defects_columns, axis=1)
        self.transverse_defects_set = self.smoothed_control_results[self.transverse_defects_columns]
        
    def load_target_variable(self, filepath):
        target = pd.read_csv(filepath)
        self.longitudinal_defects_set['y'] = 0
        self.transverse_defects_set['y_t'] = 0
        # Fill l defect
        for index, row in target[target['Тип дефекта'] == "L"].iterrows():
            self.longitudinal_defects_set["y"][int(row["Начало дефекта"]):int(row["Начало дефекта"]) + int(row["Длина дефекта"])] = 1

        for index, row in target[target['Тип дефекта'] == "T"].iterrows():
            self.transverse_defects_set["y_t"][int(row["Начало дефекта"]):15 + int(row["Начало дефекта"])] = 1
             

# Read data 

In [ ]:
test_sop_list = []
sop_files = glob.glob("Results/*test.dat")
for file_path in sop_files:
    print(file_path)
    test_sop_list.append(SOP(file_path))

for sop in test_sop_list:
    sop.read_control_results()
    sop.smooth_control_results()

# Upload target variables (anomalies) from additional file

In [ ]:
test_sop_list[0].load_target_variable("Target_/k16_test.csv")
test_sop_list[1].load_target_variable("Target_/k22_test.csv")
test_sop_list[2].load_target_variable("Target_/k3_test.csv")
test_sop_list[3].load_target_variable("Target_/k28_test.csv")
test_sop_list[4].load_target_variable("Target_/k24_test.csv")

# Feature Enginering

In [ ]:
from tsfresh.examples.robot_execution_failures import download_robot_execution_failures, \
    load_robot_execution_failures
download_robot_execution_failures()
timeseries, y = load_robot_execution_failures()
timeseries.info()

# need to understand how to do that and fix that code

# Build LSTM model for animalies detection

In [ ]:
from keras.layers import Input, Dense, LSTM, MaxPooling1D, Conv1D
from keras.models import Model
model = Sequential()
input_layer = Input(shape=(1024, 12))
conv1 = Conv1D(filters=32,
               kernel_size=8,
               strides=1,
               activation='relu',
               padding='same')(input_layer)
lstm1 = Bidirectional(LSTM(16,return_sequences=True))(conv1)
output_layer = GRU(1, activation="sigmoid", return_sequences=True)(lstm1)
model = Model(inputs=input_layer, outputs=output_layer)
# model.add(Bidirectional(LSTM(16, input_shape=(1024, 12), return_sequences=True)))
# model.add(Bidirectional(LSTM(16, return_sequences=True)))
# model.add(GRU(1, activation="sigmoid", return_sequences=True)) 
model.compile(loss="binary_crossentropy", optimizer="Adam",  metrics=['accuracy'])


### education

In [ ]:

tscv = TimeSeriesSplit(n_splits=4)
print(tscv)  

for train_index, test_index in tscv.split(train_X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = train_X[train_index], train_X[test_index]
    y_train, y_test = train_Y[train_index], train_Y[test_index]
    model.fit(X_train, y_train, epochs=10, batch_size=1)
    model.evaluate(X_test, y_test, verbose=1)
    scores = model.evaluate(X_test, y_test, verbose=1)
    print('binary_crossentropy=%f, accuracy=%f' % (scores[0],scores[1]))
    
# , validation_split=0.2